In [1]:
# Using Keras to create a 10 neurons and single layer NN
# Against the MNIST databesa

import gc
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (7,7) # Make the figures a bit bigger
nb_classes = 10

np.random.seed(42)
%matplotlib notebook

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def leaky(x):
    return K.maximum(0.3*x, x)

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

# show(X_train, y_train)

# Reshape arrays
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train original shape (60000, 28, 28)
y_train original shape (60000,)
Training matrix shape (60000, 784)
Testing matrix shape (10000, 784)


In [3]:
activations = [e_swish_2, swish, "relu", "elu", leaky] # e_swish_1, "sigmoid", "tanh"
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_2', 'swish', 'relu', 'elu', 'leaky']


In [4]:
def accuracy(y_pred, y_test):
    y_hat = np.argmax(y_pred, axis=1)
    y = np.argmax(y_test, axis=1)

    good = np.sum(np.equal(y, y_hat))
    return float(good/len(y_test))

In [5]:
def create(act):
    models = []

    sgd = SGD(lr=0.1)
    # create model
    
    model1 = Sequential()
    model1.add(Dense(200, input_dim=784, activation=act))
    model1.add(Dropout(0.2))
    model1.add(Dense(100, activation=act))
    model1.add(Dropout(0.2))
    model1.add(Dense(60, activation=act))
    model1.add(Dropout(0.2))
    model1.add(Dense(30, activation=act))
    model1.add(Dropout(0.2))
    model1.add(Dense(10, activation='softmax'))

    # Compile model
    model1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model1

In [6]:
# Configure logs
import logging

# Record settings
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="mnist_median_3_runs_log.txt",format = LOG_FORMAT, level = logging.DEBUG, filemode = "a")
logs = logging.getLogger()

In [7]:
import gc
for i,act in enumerate(activations):
    logs.info("\n \n Starting traiing for "+names[i])
    ensembler = 0
    logger = [names[i]]
    for i in range(3):
        gc.collect()
        model = create(act)
        his = model.fit(X_train, Y_train, epochs=15, batch_size=64,  verbose=1, validation_data=(X_test, Y_test))
        # Record accuracy of each model and save it
        logger.append(model.evaluate(X_test, Y_test)[1])
        # Calculate probabilities of test data and sum them toghether
        ensembler += model.predict_proba(X_test)
        # Clear session (GPU MEMORY)
        K.get_session().close()
        K.set_session(K.tf.Session())
        del model, his
        
    # Calculate the median accuracy
    ensembled = accuracy(ensembler, Y_test)
    print(ensembled)
    logs.info(str(logger))
    logs.info(str(ensembled))
    del ensembler, ensembled

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 23s 389us/step - loss: 0.4613 - acc: 0.8628 - val_loss: 0.1858 - val_acc: 0.9441
Epoch 2/15
60000/60000 [==============================] - 14s 239us/step - loss: 0.2169 - acc: 0.9396 - val_loss: 0.1310 - val_acc: 0.9609
Epoch 3/15
60000/60000 [==============================] - 14s 237us/step - loss: 0.1706 - acc: 0.9523 - val_loss: 0.1106 - val_acc: 0.9675
Epoch 4/15
60000/60000 [==============================] - 14s 238us/step - loss: 0.1393 - acc: 0.9616 - val_loss: 0.0953 - val_acc: 0.9721
Epoch 5/15
60000/60000 [==============================] - 14s 238us/step - loss: 0.1221 - acc: 0.9662 - val_loss: 0.0948 - val_acc: 0.9720
Epoch 6/15
60000/60000 [==============================] - 14s 241us/step - loss: 0.1080 - acc: 0.9704 - val_loss: 0.0887 - val_acc: 0.9733
Epoch 7/15
60000/60000 [==============================] - 14s 237us/step - loss: 0.1022 - acc: 0.9723 - val_loss: 0.

Epoch 12/15
60000/60000 [==============================] - 14s 231us/step - loss: 0.0796 - acc: 0.9769 - val_loss: 0.0750 - val_acc: 0.9772
Epoch 13/15
60000/60000 [==============================] - 14s 234us/step - loss: 0.0765 - acc: 0.9781 - val_loss: 0.0698 - val_acc: 0.9789
Epoch 14/15
60000/60000 [==============================] - 14s 234us/step - loss: 0.0701 - acc: 0.9788 - val_loss: 0.0682 - val_acc: 0.9802
Epoch 15/15
10000/10000 [==============================] - 1s 129us/step
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 20s 339us/step - loss: 0.6609 - acc: 0.7959 - val_loss: 0.2227 - val_acc: 0.9319
Epoch 2/15
60000/60000 [==============================] - 14s 235us/step - loss: 0.2775 - acc: 0.9185 - val_loss: 0.1494 - val_acc: 0.9551
Epoch 3/15
60000/60000 [==============================] - 14s 236us/step - loss: 0.2064 - acc: 0.9397 - val_loss: 0.1330 - val_acc: 0.9577
Epoch 4/15
60000/60000 [================

60000/60000 [==============================] - 13s 220us/step - loss: 0.2818 - acc: 0.9158 - val_loss: 0.1658 - val_acc: 0.9475
Epoch 3/15
60000/60000 [==============================] - 14s 239us/step - loss: 0.2298 - acc: 0.9305 - val_loss: 0.1345 - val_acc: 0.9565
Epoch 4/15
60000/60000 [==============================] - 14s 237us/step - loss: 0.2023 - acc: 0.9396 - val_loss: 0.1266 - val_acc: 0.9613
Epoch 5/15
60000/60000 [==============================] - 21s 355us/step - loss: 0.1815 - acc: 0.9460 - val_loss: 0.1106 - val_acc: 0.9632
Epoch 6/15
60000/60000 [==============================] - 29s 475us/step - loss: 0.1654 - acc: 0.9507 - val_loss: 0.1158 - val_acc: 0.9621
Epoch 7/15
60000/60000 [==============================] - 20s 330us/step - loss: 0.1546 - acc: 0.9538 - val_loss: 0.1014 - val_acc: 0.9664
Epoch 8/15
60000/60000 [==============================] - 21s 354us/step - loss: 0.1402 - acc: 0.9581 - val_loss: 0.0904 - val_acc: 0.9723
Epoch 9/15
60000/60000 [==============